<a href="https://colab.research.google.com/github/LeonardoGoncRibeiro/05_AppliedMachineLearning/blob/main/01_IntroductionRecomendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning: Introduction to recomendation systems

In this course, we will show an introduction to recomention systems using Python. We will discuss about different heuristics and algorithms that can be used to that end, and we will be able to implement an algorithm based in colaborative filters, using a Machine Learning algorithm. Finally, we will learn about the challenges in this area, and how we can improve upon our baseline algorithm.

Here, we will use a dataset from Movie Lens, with information for different movies:

In [395]:
import pandas as pd

movies  = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [396]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [397]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In the *ratings* dataframe, we have information about the user, the movie Id, and the rating given by the user to the movie. In the *movies* dataframe, we have primary key for movie Id.

# First tries: dumb recomendation systems

Before going further into the more complex models, let's try to implement some dumb recomendation systems, that try to predict and recommend movies based on very simple heuristics.



## Most popular movies

First, let's only consider the **number** of ratings each movie got. We will recommend the most popular movies, which received the highest number of ratings. Let's see what the most popular movies are:

In [398]:
popularity = ratings.movieId.value_counts( ).to_frame( ).reset_index( )
popularity.columns = ['movieId', 'Count']
popularity

,movieId,Count
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
9719,86279,1
9720,86922,1
9721,5962,1
9722,87660,1


Nice! Let's add the name of the movie to this dataframe:

In [399]:
popularity = popularity.merge(movies, left_on='movieId', right_on='movieId', how = 'left')
popularity.head(10)

,movieId,Count,title,genres
0,356,329,Forrest Gump (1994),Comedy|Drama|Romance|War
1,318,317,"Shawshank Redemption, The (1994)",Crime|Drama
2,296,307,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,593,279,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
4,2571,278,"Matrix, The (1999)",Action|Sci-Fi|Thriller
5,260,251,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
6,480,238,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller
7,110,237,Braveheart (1995),Action|Drama|War
8,589,224,Terminator 2: Judgment Day (1991),Action|Sci-Fi
9,527,220,Schindler's List (1993),Drama|War


So, the three most popular movies are Forrest Gump, Shawshank Redemption and Pulp Fiction. So, we can simply recommend the most popular movies to our user!

Nice. Now, let's get the average rating for our most popular movies:

In [400]:
ratings_grouped = ratings.groupby('movieId')
avg_rating = ratings_grouped.mean( )['rating'].to_frame( )
avg_rating.columns = ['Avg_Rating']

In [401]:
popularity = popularity.merge(avg_rating, left_on='movieId', right_index=True, how = 'left')

In [402]:
popularity = popularity[['movieId', 'title', 'genres', 'Count', 'Avg_Rating']]
popularity.head(10)

,movieId,title,genres,Count,Avg_Rating
0,356,Forrest Gump (1994),Comedy|Drama|Romance|War,329,4.164134
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,317,4.429022
2,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307,4.197068
3,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279,4.161290
4,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278,4.192446
5,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251,4.231076
6,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238,3.750000
7,110,Braveheart (1995),Action|Drama|War,237,4.031646
8,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224,3.970982
9,527,Schindler's List (1993),Drama|War,220,4.225000


Note that our first recommendation (Forrest Gump) has an average rating of 4.16, while our second has actually a highest average rating. Also, some of the most popular movies actually have a low average rating (such as Jurassic Park).

## Movies with the highest average rating

Thinking about this, another simple heuristic we can use is to simply recommend our movies based on their average rating. So, let's try this:

In [403]:
popularity.sort_values(by = 'Avg_Rating', ascending = False, inplace = True)
popularity.head(10)

,movieId,title,genres,Count,Avg_Rating
6193,3473,Jonah Who Will Be 25 in the Year 2000 (Jonas q...,Comedy,2,5.0
8703,159811,The Bremen Town Musicians (1969),Animation|Drama|Fantasy,1,5.0
8711,170597,A Plasticine Crow (1981),Animation,1,5.0
7369,120130,Into the Forest of Fireflies' Light (2011),Animation|Drama|Fantasy,1,5.0
8709,166183,Junior and Karlson (1968),Adventure|Animation|Children,1,5.0
8708,165959,Alesha Popovich and Tugarin the Dragon (2004),Animation|Comedy|Drama,1,5.0
8707,163925,"Wings, Legs and Tails (1986)",Animation|Comedy,1,5.0
9432,6983,Jane Eyre (1944),Drama|Romance,1,5.0
9433,26366,Harlan County U.S.A. (1976),Documentary,1,5.0
8706,163386,Winnie the Pooh and the Day of Concern (1972),Animation,1,5.0


This time, note that we actually have very unpopular movies. This happens because, when a movie has less votes, it is "easier" to have a very high average rating: all it takes is that the person who voted for them gave a rating 5. However, this does not really mean that the movie is really that good.

Let's try to keep the same heuristic, but let's remove those movies with less than 10 votes.

In [404]:
popularity.query('Count > 10').head(10)

,movieId,title,genres,Count,Avg_Rating
2066,1041,Secrets & Lies (1996),Drama,11,4.590909
2006,3451,Guess Who's Coming to Dinner (1967),Drama,11,4.545455
1960,1178,Paths of Glory (1957),Drama|War,12,4.541667
1284,1104,"Streetcar Named Desire, A (1951)",Drama,20,4.475000
1918,2360,"Celebration, The (Festen) (1998)",Drama,12,4.458333
1606,1217,Ran (1985),Drama|War,15,4.433333
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,317,4.429022
1671,951,His Girl Friday (1940),Comedy|Romance,14,4.392857
1375,3468,"Hustler, The (1961)",Drama,18,4.333333
943,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance,27,4.333333


It seems that 10 is still a low number. Let's test another number:

In [405]:
popularity.query('Count > 50').head(10)

,movieId,title,genres,Count,Avg_Rating
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,317,4.429022
21,858,"Godfather, The (1972)",Crime|Drama,192,4.289062
10,2959,Fight Club (1999),Action|Crime|Drama|Thriller,218,4.272936
364,1276,Cool Hand Luke (1967),Drama,57,4.271930
143,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97,4.268041
189,904,Rear Window (1954),Mystery|Thriller,84,4.261905
73,1221,"Godfather: Part II, The (1974)",Crime|Drama,129,4.259690
113,48516,"Departed, The (2006)",Crime|Drama|Thriller,107,4.252336
76,1213,Goodfellas (1990),Crime|Drama,126,4.250000
134,912,Casablanca (1942),Drama|Romance,100,4.240000


Ok! Now, this seems like a good recommendation, which takes into account both the average rating and the number of votes.

Note that, here, we are considering that we know **nothing** about the user we want to give the recommendation. Sometimes, however, we know, for instance, the movie that person saw. Thus, this data can help us to guide the recommendation system towards movies closer to what this person likes. Let's see how we can do this.

# Recommendation based on the movie genre

Usually, recommendation system take into consideration your previous choices. For instance, let's see userId 329:

In [406]:
ratings.query('userId == 329').head(5)

,userId,movieId,rating,timestamp
50925,329,50,2.0,1523468332
50926,329,318,2.0,1523468325
50927,329,593,2.0,1523468340
50928,329,750,3.0,1523468346
50929,329,778,4.5,1523468384


We know, for instance, that this person watched movies 50, 318, 593, 750, and 778, which are:

In [407]:
movies.query('movieId in [50, 318, 593, 750, 778]')

,movieId,title,genres
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
277,318,"Shawshank Redemption, The (1994)",Crime|Drama
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
602,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War
613,778,Trainspotting (1996),Comedy|Crime|Drama


Nice! Notice that we have the genres of these movies. What if we tried to recommend movies with the same genres? Let's try to get the movies from the Crime|Drama genre:

In [408]:
recommendation_1 = popularity.query("genres == 'Crime|Drama' and Count > 50").sort_values(by = 'Avg_Rating', ascending = False).set_index('movieId')
recommendation_1

,title,genres,Count,Avg_Rating
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317,4.429022
858,"Godfather, The (1972)",Crime|Drama,192,4.289062
1221,"Godfather: Part II, The (1974)",Crime|Drama,129,4.259690
1213,Goodfellas (1990),Crime|Drama,126,4.250000
2329,American History X (1998),Crime|Drama,129,4.217054
3147,"Green Mile, The (1999)",Crime|Drama,111,4.148649
16,Casino (1995),Crime|Drama,82,3.926829
5989,Catch Me If You Can (2002),Crime|Drama,115,3.921739
55820,No Country for Old Men (2007),Crime|Drama,64,3.898438


Nice! Also, let's remove the movies this person has already saw:

In [409]:
recommendation_1.drop(ratings.query('userId == 329').movieId, errors = 'ignore', inplace = True)
recommendation_1.head( )

,title,genres,Count,Avg_Rating
movieId,,,,
1221,"Godfather: Part II, The (1974)",Crime|Drama,129,4.259690
1213,Goodfellas (1990),Crime|Drama,126,4.250000
2329,American History X (1998),Crime|Drama,129,4.217054
3147,"Green Mile, The (1999)",Crime|Drama,111,4.148649
16,Casino (1995),Crime|Drama,82,3.926829


Nice! Based on this, we can recommend the Godfather: Part II to our user. Note that, here, we considered some information about our user to make this recommendation. Thus, knowing a popular genre my user likes, we can recommend other movies from that genre.

Another idea is to draw a *profile* for our user, based on the latest movies this user saw. 

# Implementing a Machine Learning algorithm

In this section, we will start to implement a very popular machine learning algorithm: K-Nearest Neighbors (K-NN). We will perform each step of the algorithm and, in the end, use it to find good recommendations.

## Evaluating the Euclidean distance

A very good idea to find good recommendations is to try to find which users are similar to a given user. And how do we define similarity?

The Euclidean distance is a very simple way to find similar data points. The higher the distance, the lower the similarity. Let's say that we have customers A and B, and their ratings for two different movies were:

*   A: 4 and 4.5
*   B: 5 and 5

We can evaluate their similarity by doing:

\begin{equation}
d = \sqrt{(5 - 3)^2 + (5 - 4)^2}
\end{equation}

which is basically the two-dimensional euclidean distance. For higher distances, our users are less similar! We can evaluate the Euclidean distance by the norm of the difference between two vectors:



In [410]:
import numpy as np

person_A = np.array([4, 4.5])
person_B = np.array([5, 5])
person_C = np.array([3.5, 4.5])

In [411]:
# Evaluating the distance between A and B

np.linalg.norm(person_B - person_A)

1.118033988749895

In [412]:
# Evaluating the distance between B and C

np.linalg.norm(person_C - person_B)

1.5811388300841898

So, by the Euclidean distance, we can infer that person B is more similar to person A than to person C!

## Evaluating the Euclidean distance between all individuals and a given user

Finally, let's evaluate the distance from the users in our datasets. Let's get the ratings for user 1:

In [413]:
user1_ratings = ratings.query('userId == 1')[['movieId', 'rating']].set_index('movieId')
user1_ratings

,rating
movieId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


Nice! To simplify things up, let's define a function to get this dataframe:

In [414]:
def GetUserRatings(user):
  user_ratings = ratings.query('userId == %d' % user)[['movieId', 'rating']].set_index('movieId')
  return user_ratings

Now, if we get the ratings for user 1:

In [415]:
GetUserRatings(1)

,rating
movieId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


Nice! We got the same dataframe. Now, let's get the ratings for user 1 and 4.

In [416]:
user1_ratings = GetUserRatings(1)
user4_ratings = GetUserRatings(4)

Note that we need to compare movie 1 with movie 1, movie 2 with movie 2, and so on. To join these dataframe correctly, we can do:

In [417]:
table_1_4 = user1_ratings.join(user4_ratings, lsuffix = '_l', rsuffix = '_r').dropna( )
table_1_4

,rating_l,rating_r
movieId,,
47,5.0,2.0
235,4.0,2.0
260,5.0,5.0
296,3.0,1.0
441,4.0,1.0
457,5.0,5.0
553,5.0,2.0
593,4.0,5.0
608,5.0,5.0


Nice! Now, we have the ratings for each user for the movies that both users rated. Now, let's get the Euclidean distance between them:

In [418]:
euc_dist_1_4 = np.linalg.norm(table_1_4.rating_l - table_1_4.rating_r)
euc_dist_1_4

11.135528725660043

Nice! Now, let's implement all of this in a single function:

In [419]:
def GetEucDistanceUsers(user1, user2):
  left  = GetUserRatings(user1)
  right = GetUserRatings(user2)
  table = left.join(right, lsuffix = '_l', rsuffix = '_r').dropna( )
  euc_dist = np.linalg.norm(table.rating_l - table.rating_r)
  return euc_dist

In [420]:
GetEucDistanceUsers(1, 4)

11.135528725660043

Great! We got exactly the same result.

Now, let's try to understand which users are closest to user 1:

In [421]:
user_list = ratings.userId.unique( )

In [422]:
def GetEucDistAll(user, list_users):
  dist = [GetEucDistanceUsers(user, user_i) for user_i in list_users]
  return dist

In [423]:
d = GetEucDistAll(1, user_list)

Nice! Now, let's create a dataframe with this information:

In [424]:
similarity_user = pd.DataFrame({'user' : user_list, 'distance' : d})
similarity_user

,user,distance
0,1,0.000000
1,2,1.414214
2,3,8.200610
3,4,11.135529
4,5,3.741657
...,...,...
605,606,11.510864
606,607,9.899495
607,608,18.241436
608,609,3.162278


Great! Let's update our function to create this dataframe:

In [425]:
def GetEucDistAll(user, list_users):
  dist = [GetEucDistanceUsers(user, user_i) for user_i in list_users]
  similarity_user = pd.DataFrame({'you' : user, 'user' : list_users, 'distance' : dist})
  return similarity_user

In [426]:
similarity_user = GetEucDistAll(1, user_list)
similarity_user

,you,user,distance
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
...,...,...,...
605,1,606,11.510864
606,1,607,9.899495
607,1,608,18.241436
608,1,609,3.162278


## Evaluating the most similar users

Nice! Now, let's order this dataframe:

In [427]:
similarity_user.sort_values(by = 'distance')

,you,user,distance
0,1,1,0.000000
577,1,578,0.000000
76,1,77,0.000000
84,1,85,0.000000
174,1,175,0.000000
...,...,...,...
473,1,474,18.594354
159,1,160,18.794946
216,1,217,19.646883
598,1,599,19.665960


Nice! However, there are some strange things here: note that the distance was null for users 578, 77, 85, 175. Did these users really rated the movies in the same way as user 1? Let's see:

In [428]:
r1   = GetUserRatings(1)
r578 = GetUserRatings(578)

In [429]:
table_check = r1.join(r578, lsuffix = '_l', rsuffix = '_r').dropna( )
table_check

,rating_l,rating_r
movieId,,


Note that, actually, these users never saw the same movies. Then, their distance was 0!

Before evaluating the distance, we should remove the cases where there are less than $n$ movies in common. Another way is to penalize these cases, giving them a very high distance. Let's update our function:

In [430]:
def GetEucDistanceUsers(user1, user2, min_len = 5):
  left  = GetUserRatings(user1)
  right = GetUserRatings(user2)
  table = left.join(right, lsuffix = '_l', rsuffix = '_r').dropna( )

  if table.shape[0] < min_len:
    return 1e5

  euc_dist = np.linalg.norm(table.rating_l - table.rating_r)
  return euc_dist

Now, let's test it again:

In [431]:
similarity_user = GetEucDistAll(1, user_list)

In [432]:
similarity_user.sort_values(by = 'distance')

,you,user,distance
0,1,1,0.000000
76,1,77,0.000000
510,1,511,0.500000
365,1,366,0.707107
522,1,523,1.000000
...,...,...,...
189,1,190,100000.000000
59,1,60,100000.000000
575,1,576,100000.000000
544,1,545,100000.000000


Nice! Now, let's test user 77, which still got a 0 distance:

In [433]:
r1  = GetUserRatings(1)
r77 = GetUserRatings(77)

In [434]:
table_check = r1.join(r77, lsuffix = '_l', rsuffix = '_r').dropna( )
table_check

,rating_l,rating_r
movieId,,
260,5.0,5.0
1196,5.0,5.0
1198,5.0,5.0
1210,5.0,5.0
2571,5.0,5.0
3578,5.0,5.0


Nice! Now, these users have very few movies, but, indeed, they rated all of these movies equally!

Finally, let's implement a function to get the most similar users (removing you from the list):

In [435]:
def GetClosestTo(user, list_users):
  similarity_user = GetEucDistAll(user, list_users)
  similarity_user = similarity_user.set_index('user').drop(user)
  return similarity_user.sort_values(by = 'distance')

In [436]:
GetClosestTo(1, user_list)

,you,distance
user,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
258,1,1.000000
49,1,1.000000
...,...,...
315,1,100000.000000
291,1,100000.000000
285,1,100000.000000


Great! We got the same result.

## Testing our system

Ok. Before testing our algorithm, let's make some small modifications on our functions, to be sure that we will not store the penalized results in our final dataframe:

In [437]:
def GetEucDistanceUsers(user1, user2, min_len = 5):
  left  = GetUserRatings(user1)
  right = GetUserRatings(user2)
  table = left.join(right, lsuffix = '_l', rsuffix = '_r').dropna( )

  if table.shape[0] < min_len:
    return None

  euc_dist = np.linalg.norm(table.rating_l - table.rating_r)
  return euc_dist

Then, later, we can filter our result:

In [438]:
def GetEucDistAll(user, list_users):
  dist = [GetEucDistanceUsers(user, user_i) for user_i in list_users]

  similarity_user = pd.DataFrame({'you' : user, 'user' : list_users, 'distance' : dist}).dropna( )
  return similarity_user

Now, if we do:

In [439]:
GetClosestTo(1, user_list)

,you,distance
user,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
9,1,1.000000
49,1,1.000000
...,...,...
474,1,18.594354
160,1,18.794946
217,1,19.646883


Nice. From now on, we will work with a reduce list of users, due to high running times. Thus, let's update our function again:

In [440]:
def GetClosestTo(user, list_users, n_users_to_analyze = None):
  if (n_users_to_analyze) and (n_users_to_analyze < len(list_users)):
    list_users = list_users[:n_users_to_analyze]
  similarity_user = GetEucDistAll(user, list_users)
  similarity_user = similarity_user.set_index('user').drop(user)
  return similarity_user.sort_values(by = 'distance')

In [441]:
GetClosestTo(1, user_list, n_users_to_analyze = 50)

,you,distance
user,,
49,1,1.000000
9,1,1.000000
25,1,1.414214
13,1,1.414214
30,1,1.802776
26,1,2.236068
35,1,2.236068
46,1,3.316625
5,1,3.741657


Nice! Everything seems to be working out. If we want to get the most similar individual, we can do:

In [442]:
most_similar = GetClosestTo(1, user_list, n_users_to_analyze = 50).iloc[0].name
most_similar

49

Nice! So, user 49 is the most similar from user 1 (from the first 50 users). Which movies did user 49 saw? 

In [443]:
GetUserRatings(49)

,rating
movieId,
110,4.0
318,4.0
356,4.0
527,4.5
1097,4.5
1200,4.5
1214,4.0
2028,4.5
2571,4.5


Nice! And which movies did the user 49 see, but the user 1 did not?

In [444]:
r1  = GetUserRatings(1)
r49 = GetUserRatings(49)

missing_movies = r49.drop(r1.index, errors = 'ignore')
missing_movies

,rating
movieId,
318,4.0
1200,4.5
4022,4.5
5218,4.0
6377,4.0
47099,4.5
70286,4.0
79091,4.0
79132,4.5


Great! Now, let's order these movies by rating:

In [445]:
missing_movies.sort_values(by = 'rating', ascending = False, inplace = True)
missing_movies

,rating
movieId,
1200,4.5
4022,4.5
47099,4.5
79132,4.5
109487,4.5
139385,4.5
168252,4.5
318,4.0
5218,4.0


Nice! So, using this criterion, we would suggest movieId = 1200 to user 1, which is:

In [446]:
movies.query('movieId == 1200')

,movieId,title,genres
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi


Aliens!

Finally, let's group everything in a user-defined function:

In [447]:
def FindSuggestionID(user, n_analysis, n_recommend):
  most_similar = GetClosestTo(user, user_list, n_users_to_analyze = n_analysis).iloc[0].name
  r_user = GetUserRatings(user)
  r_sim  = GetUserRatings(most_similar)

  missing_movies = r_sim.drop(r_user.index, errors = 'ignore')
  missing_movies.sort_values(by = 'rating', ascending = False, inplace = True)
  missing_movies = missing_movies.merge(popularity[['movieId', 'title', 'Count', 'Avg_Rating']], left_index = True, right_on = 'movieId', how = 'left').head(n_recommend)
  return missing_movies

In [448]:
FindSuggestionID(1, 50, 5)

,rating,movieId,title,Count,Avg_Rating
75,4.5,1200,Aliens (1986),126,3.964286
137,4.5,4022,Cast Away (2000),100,3.700000
498,4.5,47099,"Pursuit of Happyness, The (2006)",46,3.793478
51,4.5,79132,Inception (2010),143,4.066434
240,4.5,109487,Interstellar (2014),73,3.993151


Nice! Now, we got the ID for our best suggestions, as well as the name of the movies.

Now, let's try to get the recommendation considering the entire list of users:

In [449]:
FindSuggestionID(1, 610, 5)

,rating,movieId,title,Count,Avg_Rating
212,5.0,8636,Spider-Man 2 (2004),79,3.803797
44,5.0,58559,"Dark Knight, The (2008)",149,4.238255
93,5.0,33794,Batman Begins (2005),116,3.862069
19,5.0,4993,"Lord of the Rings: The Fellowship of the Ring,...",198,4.106061
82,5.0,5349,Spider-Man (2002),122,3.540984


## Finishing our implementation of K-NN

Nice! But should we consider only the most similar user? Why not consider a group of similar users? That is the idea of the very popular K-Nearest Neighbors algorithm. 

So, let's try to update our function:

In [450]:
def FindSuggestionID(user, n_analysis, n_recommend, K):
  most_similar = GetClosestTo(user, user_list, n_users_to_analyze = n_analysis).iloc[:K].index
  r_user = GetUserRatings(user)
  r_sim = ratings.set_index('userId').loc[most_similar].groupby('movieId').mean( )['rating'].to_frame( )

  missing_movies = r_sim.drop(r_user.index, errors = 'ignore')
  missing_movies.sort_values(by = 'rating', ascending = False, inplace = True)
  missing_movies = missing_movies.merge(popularity[['movieId', 'title', 'Count', 'Avg_Rating']], left_index = True, right_on = 'movieId', how = 'left').head(n_recommend)
  return missing_movies

Note that, here, we are grabbing the missing movies considering the $K$ best users. The final rating is the mean from these $K$ users. Then, we sort the missing movies by rating. 

Now, let's test our function:

In [451]:
FindSuggestionID(1, 50, 5, 3)

,rating,movieId,title,Count,Avg_Rating
1902,5.0,187593,Deadpool 2 (2018),12,3.875000
443,5.0,116797,The Imitation Game (2014),50,4.020000
26,5.0,7153,"Lord of the Rings: The Return of the King, The...",185,4.118919
287,5.0,5481,Austin Powers in Goldmember (2002),65,2.846154
44,5.0,58559,"Dark Knight, The (2008)",149,4.238255


Nice! Based on the three most similar users, we would recommend these movies to user 1. Note that, here, we only considered the 50 first users. Let's now consider the entire user list:

In [452]:
FindSuggestionID(1, 610, 5, 3)

,rating,movieId,title,Count,Avg_Rating
1,5.0,318,"Shawshank Redemption, The (1994)",317,4.429022
103,5.0,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang lon...",110,3.836364
842,5.0,139385,The Revenant (2015),31,3.903226
691,5.0,92259,Intouchables (2011),37,4.108108
212,5.0,8636,Spider-Man 2 (2004),79,3.803797


Nice! We managed to implement the K-Nearest Neighbors algorithm!

# K-Nearest Neighbors algorithm using Surprise

Note that, here, we implemented a variation of the K-NN algorithm, which can be used for recommendation systems. The K-Nearest Neighbors algorithm considers the distance from each data point to the others, and makes a prediction based on their $K$ nearest neighbors (which are more similar) to it. Here, the prediction is the recommendation of a given movie.

More information about the K-Nearest Neighbors algorithm can be found in:

https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm

Also, for more information about recommendation systems, we can go to:

https://en.wikipedia.org/wiki/Recommender_system

A very well-known Python library to work with recommendation systems is Surprise (Simple Python RecommendatIon System Engine):

In [453]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [454]:
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline
from surprise.model_selection import KFold
from surprise import accuracy

In [455]:
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
anti_set = data.build_full_trainset().build_anti_testset()

Here, the antiset is a set of those user-item pairs for which there is no rating in the original dataset. In the antiset, we will evaluate an average rating for these non-existent combinations.

In [456]:
movies_list = ratings.movieId.unique( )
users_list  = ratings.userId.unique( )

Now, let's use the K-NN:

In [457]:
kf = KFold(n_splits=3) # We will test 3 different train test splits, and get the best model out of those

knn_alg = KNNBasic()
best_algo = None
best_rmse = 1000.0
best_pred = None

for trainset, testset in kf.split(data):
    # train and test algorithm.
    knn_alg.fit(trainset)
    predictions = knn_alg.test(testset)

    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_algo = knn_alg
        best_pred = predictions

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9593
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9509
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9635


Here, we tested three different K-NN algorithms (changing the train-test splits), and we save the best algorithm out of those. Note that, here, we used an algorithm named KNNBasic, where, instead of the average rating of each neighbor, we will use the similarity to give different weights to each rating. We can also test different algorithms, such as the KNN with means or the KNN Baseline:

In [458]:
kf = KFold(n_splits=5)
sim_options = {'name':'cosine'}
algo = KNNWithMeans(sim_options = sim_options)
best_algo = None
best_rmse = 1000.0
best_pred = None
for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_algo = algo
        best_rmse= rmse
        best_pred = predictions
print(best_rmse)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9136
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8971
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8923
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9085
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8938
0.8922872160501474


In [459]:
kf = KFold(n_splits=3)
algo = KNNBaseline(k=3)
best_algo = None
best_rmse = 1000.0
best_pred = None
for trainset, testset in kf.split(data):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_rmse = rmse
        best_algo = algo
        best_pred = predictions

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9425
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9437
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9431


More about these algorithms can be found in:

https://surprise.readthedocs.io/en/stable/knn_inspired.html

We can get the best prediction from:

In [460]:
best_pred[:5]

[Prediction(uid=453, iid=3908, r_ui=1.0, est=2.8152361345502244, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=135, iid=1407, r_ui=4.0, est=3.6020751521628473, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=41, iid=55765, r_ui=2.0, est=3.493823976912312, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=392, iid=2706, r_ui=5.0, est=3.485814682890873, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=166, iid=648, r_ui=3.0, est=3.42426068266608, details={'actual_k': 3, 'was_impossible': False})]

Which gives the user id, item id, the rating, and the estimated rating. Let's get a dataframe:

In [461]:
pred_df = pd.DataFrame(best_pred).merge(ratings , left_on = ['uid', 'iid'], right_on = ['userId', 'movieId'])
pred_df[['uid', 'iid', 'userId', 'movieId', 'est','rating']]

,uid,iid,userId,movieId,est,rating
0,453,3908,453,3908,2.815236,1.0
1,135,1407,135,1407,3.602075,4.0
2,41,55765,41,55765,3.493824,2.0
3,392,2706,392,2706,3.485815,5.0
4,166,648,166,648,3.424261,3.0
...,...,...,...,...,...,...
33607,186,5499,186,5499,3.893636,3.0
33608,608,1573,608,1573,2.561495,2.5
33609,140,2269,140,2269,2.870490,3.0
33610,254,50,254,50,4.713360,4.5


Now, let's get the prediction on our antiset for user 1:

In [462]:
anti_set_df = pd.DataFrame(anti_set, columns = ['userId', 'movieId', 'avg_rating'])

In [463]:
anti_set_df_1 = anti_set_df.query('userId == 1')

In [464]:
records = anti_set_df_1.to_records(index=False)
anti_set_1 = list(records)

In [466]:
anti_pre = best_algo.test(anti_set_1)
pred_df = pd.DataFrame(anti_pre).merge(movies , left_on = ['iid'], right_on = ['movieId'])[['uid', 'movieId', 'title', 'est']]

pred_df = pred_df.sort_values(by = 'est', ascending = False)

In [467]:
pred_df

,uid,movieId,title,est
4746,1,6669,Ikiru (1952),5.000000
3729,1,136503,Tom and Jerry: Shiver Me Whiskers (2006),5.000000
3777,1,164226,Maximum Ride (2016),5.000000
3772,1,158882,All Yours (2016),5.000000
3771,1,158398,World of Glory (1991),5.000000
...,...,...,...,...
3965,1,77427,"Human Centipede, The (First Sequence) (2009)",1.011137
2748,1,59306,Prom Night (2008),0.994549
8711,1,7742,Baxter (1989),0.704751
8696,1,4794,Opera (1987),0.704751


So, based on the estimated rating, we can recommend a movie to our user 1! To finish, let's get only those movies with more than 50 votes:

In [470]:
pred_df = pred_df.merge(popularity[['movieId', 'Count']], left_on = 'movieId', right_on = 'movieId', how = 'left')

In [472]:
pred_df.query('Count > 50')

,uid,movieId,title,est,Count
15,1,68954,Up (2009),5.000000,105
124,1,1028,Mary Poppins (1964),5.000000,71
152,1,8961,"Incredibles, The (2004)",5.000000,125
178,1,4995,"Beautiful Mind, A (2001)",5.000000,123
180,1,5349,Spider-Man (2002),5.000000,122
...,...,...,...,...,...
9016,1,172,Johnny Mnemonic (1995),2.784734,53
9047,1,420,Beverly Hills Cop III (1994),2.730202,59
9075,1,4370,A.I. Artificial Intelligence (2001),2.702082,56
9217,1,788,"Nutty Professor, The (1996)",2.478646,82


Nice!